## Data Preperation 
### data set: https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks

In [3]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

In [226]:
import warnings
warnings.filterwarnings("ignore")

In [227]:
%matplotlib inline

In [228]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [229]:
genres_df= pd.read_csv("/Users/victoriapuck-karam/Documents/musicdata/genres_v2.csv")
genres_df # with genre information 

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.059800,0.013400,0.0556,0.3890,156.985,audio_features,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.401000,0.000000,0.1180,0.1240,115.080,audio_features,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.013800,0.000004,0.3720,0.0391,218.050,audio_features,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN
3,0.476,0.781,0,-4.710,1,0.1030,0.023700,0.000000,0.1140,0.1750,186.948,audio_features,0VSXnJqQkwuH2ei1nOQ1nu,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN
4,0.798,0.624,2,-7.668,1,0.2930,0.217000,0.000000,0.1660,0.5910,147.988,audio_features,4jCeguq9rMTlbMmPHuO7S3,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42300,0.528,0.693,4,-5.148,1,0.0304,0.031500,0.000345,0.1210,0.3940,150.013,audio_features,46bXU7Sgj7104ZoXxzz9tM,spotify:track:46bXU7Sgj7104ZoXxzz9tM,https://api.spotify.com/v1/tracks/46bXU7Sgj710...,https://api.spotify.com/v1/audio-analysis/46bX...,269208,4,hardstyle,NaN,20995.0,Euphoric Hardstyle
42301,0.517,0.768,0,-7.922,0,0.0479,0.022500,0.000018,0.2050,0.3830,149.928,audio_features,0he2ViGMUO3ajKTxLOfWVT,spotify:track:0he2ViGMUO3ajKTxLOfWVT,https://api.spotify.com/v1/tracks/0he2ViGMUO3a...,https://api.spotify.com/v1/audio-analysis/0he2...,210112,4,hardstyle,NaN,20996.0,Greatest Hardstyle Playlist
42302,0.361,0.821,8,-3.102,1,0.0505,0.026000,0.000242,0.3850,0.1240,154.935,audio_features,72DAt9Lbpy9EUS29OzQLob,spotify:track:72DAt9Lbpy9EUS29OzQLob,https://api.spotify.com/v1/tracks/72DAt9Lbpy9E...,https://api.spotify.com/v1/audio-analysis/72DA...,234823,4,hardstyle,NaN,20997.0,Best of Hardstyle 2020
42303,0.477,0.921,6,-4.777,0,0.0392,0.000551,0.029600,0.0575,0.4880,150.042,audio_features,6HXgExFVuE1c3cq9QjFCcU,spotify:track:6HXgExFVuE1c3cq9QjFCcU,https://api.spotify.com/v1/tracks/6HXgExFVuE1c...,https://api.spotify.com/v1/audio-analysis/6HXg...,323200,4,hardstyle,NaN,20998.0,Euphoric Hardstyle


In [230]:
genres_df['genre'].values[42205]

'hardstyle'

In [231]:
genres_df.dtypes

danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
type                 object
id                   object
uri                  object
track_href           object
analysis_url         object
duration_ms           int64
time_signature        int64
genre                object
song_name            object
Unnamed: 0          float64
title                object
dtype: object

In [232]:
tracks_df= pd.read_csv("/Users/victoriapuck-karam/Documents/musicdata/tracks.csv")
tracks_df #without genre information 

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4


In [233]:
tracks_df.dtypes


id                   object
name                 object
popularity            int64
duration_ms           int64
explicit              int64
artists              object
id_artists           object
release_date         object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object

In [234]:
genres_df['id']= genres_df['id'].convert_dtypes()# with genre information 
tracks_df['id']= tracks_df['id'].convert_dtypes() #without genre information 

### Formatting of artist data, various forms including lists 

In [235]:
genres_df['genre'].values[0]

'Dark Trap'

In [236]:
tracks_df['artists'].values[0]

"['Uli']"

In [237]:
tracks_df['artists'].values[0][0]

'['

In [238]:
tracks_df['artists_upd_v1'] = tracks_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))


In [239]:
tracks_df['artists_upd_v1'].values[0][0]

'Uli'

In [240]:
tracks_df[tracks_df['artists_upd_v1'].apply(lambda x: not x)].head(5)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1
164,1xEEYhWxT4WhDQdxfPCT8D,Snake Rag,20,194533,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.708,0.361,0,-11.764,0,0.0441,0.994,0.883,0.103,0.902,105.695,4,[]
170,3rauXVLOOM5BlxWqUcDpkg,Chimes Blues,14,170827,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.546,0.189,0,-15.984,1,0.0581,0.996,0.908,0.339,0.554,80.318,4,[]
172,1UdqHVRFYMZKU2Q7xkLtYc,Pickin' On Your Baby,11,197493,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.520,0.153,0,-14.042,1,0.0440,0.995,0.131,0.353,0.319,102.937,4,[]
174,0Vl2DO5U6FjgBpzCtBN3OA,Everybody Loves My Baby,10,152507,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.514,0.193,2,-13.920,0,0.2380,0.996,0.199,0.248,0.665,180.674,4,[]
180,5SvyP1ZeJX1jA7AOZD08NA,Tears,10,187227,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.359,0.357,3,-11.810,1,0.0511,0.994,0.819,0.290,0.753,205.053,4,[]


In [241]:
tracks_df['artists_upd_v2'] = tracks_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))

In [242]:
tracks_df['artists_upd'] = np.where(tracks_df['artists_upd_v1'].apply(lambda x: not x), tracks_df['artists_upd_v2'], tracks_df['artists_upd_v1'] )


In [243]:
tracks_df.convert_dtypes()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,3,[Uli],[],[Uli]
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.263,0,-22.136,1,0.957,0.797,0.0,0.148,0.655,102.009,1,[Fernando Pessoa],[],[Fernando Pessoa]
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.177,1,-21.18,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,[Ignacio Corsini],[],[Ignacio Corsini]
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918,0.104,0.397,169.98,3,[Ignacio Corsini],[],[Ignacio Corsini]
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.158,3,-16.9,0,0.039,0.989,0.13,0.311,0.196,103.22,4,[Dick Haymes],[],[Dick Haymes]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.56,0.518,0,-7.471,0,0.0292,0.785,0.0,0.0648,0.211,131.896,4,[阿YueYue],[],[阿YueYue]
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.663,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.686,150.091,4,[ROLE MODEL],[],[ROLE MODEL]
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.314,7,-12.823,0,0.0408,0.895,0.00015,0.0874,0.0663,145.095,4,[FINNEAS],[],[FINNEAS]
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.615,10,-6.212,1,0.0345,0.206,0.000003,0.305,0.438,90.029,4,"[Gentle Bones, Clara Benin]",[],"[Gentle Bones, Clara Benin]"


In [244]:
tracks_df.dtypes

id                   string
name                 object
popularity            int64
duration_ms           int64
explicit              int64
artists              object
id_artists           object
release_date         object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
artists_upd_v1       object
artists_upd_v2       object
artists_upd          object
dtype: object

In [245]:
tracks_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,[Uli],[],[Uli]
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,[Fernando Pessoa],[],[Fernando Pessoa]
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,[Ignacio Corsini],[],[Ignacio Corsini]
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,[Ignacio Corsini],[],[Ignacio Corsini]
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,[Dick Haymes],[],[Dick Haymes]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,[阿YueYue],[],[阿YueYue]
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,[ROLE MODEL],[],[ROLE MODEL]
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,[FINNEAS],[],[FINNEAS]
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,"[Gentle Bones, Clara Benin]",[],"[Gentle Bones, Clara Benin]"


In [246]:
tracks_df['artists_upd3']= tracks_df['artists_upd'].astype(str)

In [247]:
tracks_df['names_strs']=tracks_df['name'].astype(str)

In [248]:
tracks_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_upd3,names_strs
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,[Uli],[],[Uli],['Uli'],Carve
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,[Fernando Pessoa],[],[Fernando Pessoa],['Fernando Pessoa'],Capítulo 2.16 - Banquero Anarquista
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],Vivo para Quererte - Remasterizado
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],El Prisionero - Remasterizado
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,[Dick Haymes],[],[Dick Haymes],['Dick Haymes'],Lady of the Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,[阿YueYue],[],[阿YueYue],['阿YueYue'],云与海
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,[ROLE MODEL],[],[ROLE MODEL],['ROLE MODEL'],blind
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,[FINNEAS],[],[FINNEAS],['FINNEAS'],What They'll Say About Us
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,"[Gentle Bones, Clara Benin]",[],"[Gentle Bones, Clara Benin]","['Gentle Bones', 'Clara Benin']",A Day At A Time


In [249]:
tracks_df['artists_song'] = tracks_df.apply(lambda row: row['artists_upd3']+row['names_strs'],axis = 1)

In [250]:
tracks_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_upd3,names_strs,artists_song
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,[Uli],[],[Uli],['Uli'],Carve,['Uli']Carve
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,[Fernando Pessoa],[],[Fernando Pessoa],['Fernando Pessoa'],Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa']Capítulo 2.16 - Banquero An...
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],Vivo para Quererte - Remasterizado,['Ignacio Corsini']Vivo para Quererte - Remast...
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],El Prisionero - Remasterizado,['Ignacio Corsini']El Prisionero - Remasterizado
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,[Dick Haymes],[],[Dick Haymes],['Dick Haymes'],Lady of the Evening,['Dick Haymes']Lady of the Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,[阿YueYue],[],[阿YueYue],['阿YueYue'],云与海,['阿YueYue']云与海
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,[ROLE MODEL],[],[ROLE MODEL],['ROLE MODEL'],blind,['ROLE MODEL']blind
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,[FINNEAS],[],[FINNEAS],['FINNEAS'],What They'll Say About Us,['FINNEAS']What They'll Say About Us
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,"[Gentle Bones, Clara Benin]",[],"[Gentle Bones, Clara Benin]","['Gentle Bones', 'Clara Benin']",A Day At A Time,"['Gentle Bones', 'Clara Benin']A Day At A Time"


#### Made new variable to identify all unique song name and artist names to prevent repeated songs 

In [251]:
tracks_df.drop_duplicates('artists_song',inplace = True)

In [252]:
tracks_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_upd3,names_strs,artists_song
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,[Uli],[],[Uli],['Uli'],Carve,['Uli']Carve
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,[Fernando Pessoa],[],[Fernando Pessoa],['Fernando Pessoa'],Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa']Capítulo 2.16 - Banquero An...
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],Vivo para Quererte - Remasterizado,['Ignacio Corsini']Vivo para Quererte - Remast...
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],El Prisionero - Remasterizado,['Ignacio Corsini']El Prisionero - Remasterizado
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,[Dick Haymes],[],[Dick Haymes],['Dick Haymes'],Lady of the Evening,['Dick Haymes']Lady of the Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,[阿YueYue],[],[阿YueYue],['阿YueYue'],云与海,['阿YueYue']云与海
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,[ROLE MODEL],[],[ROLE MODEL],['ROLE MODEL'],blind,['ROLE MODEL']blind
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,[FINNEAS],[],[FINNEAS],['FINNEAS'],What They'll Say About Us,['FINNEAS']What They'll Say About Us
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,"[Gentle Bones, Clara Benin]",[],"[Gentle Bones, Clara Benin]","['Gentle Bones', 'Clara Benin']",A Day At A Time,"['Gentle Bones', 'Clara Benin']A Day At A Time"


In [253]:
artists_exploded = tracks_df[['artists_upd','id']].explode('artists_upd')

In [254]:
artists_exploded

,artists_upd,id
0,Uli,35iwgR4jXetI318WEWsa1Q
1,Fernando Pessoa,021ht4sdgPcrDgSk7JTbKY
2,Ignacio Corsini,07A5yehtSnoedViJAZkNnc
3,Ignacio Corsini,08FmqUhxtyLTn6pAh6bk45
4,Dick Haymes,08y9GfoqCWfOGsKdwojr5e
...,...,...
586668,ROLE MODEL,0NuWgxEp51CutD2pJoF4OM
586669,FINNEAS,27Y1N4Q4U3EfDU5Ubw8ws2
586670,Gentle Bones,45XJsGpFTyzbzeWK8VzR8S
586670,Clara Benin,45XJsGpFTyzbzeWK8VzR8S


## modifications: alterations in data normalization

In [255]:
artists_exploded_enriched = artists_exploded.merge(genres_df, how = 'left', left_on = 'id',right_on = 'id')

In [256]:
artists_exploded_enriched

,artists_upd,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,Uli,35iwgR4jXetI318WEWsa1Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fernando Pessoa,021ht4sdgPcrDgSk7JTbKY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ignacio Corsini,07A5yehtSnoedViJAZkNnc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ignacio Corsini,08FmqUhxtyLTn6pAh6bk45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dick Haymes,08y9GfoqCWfOGsKdwojr5e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683355,ROLE MODEL,0NuWgxEp51CutD2pJoF4OM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683356,FINNEAS,27Y1N4Q4U3EfDU5Ubw8ws2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683357,Gentle Bones,45XJsGpFTyzbzeWK8VzR8S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683358,Clara Benin,45XJsGpFTyzbzeWK8VzR8S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [257]:
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genre.isnull()]

In [258]:
artists_exploded_enriched_nonnull

,artists_upd,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
52733,Johnny Cash,6YffUZJ2R06kyxyK6onezL,0.659,0.585,7.0,-8.189,1.0,0.0288,0.623000,0.000213,0.3480,0.784,208.222,audio_features,spotify:track:6YffUZJ2R06kyxyK6onezL,https://api.spotify.com/v1/tracks/6YffUZJ2R06k...,https://api.spotify.com/v1/audio-analysis/6Yff...,158427.0,4.0,RnB,Ring of Fire,NaN,NaN
57144,Jimi Hendrix,0wJoRiX5K5BxlqZTolB2LD,0.533,0.905,2.0,-5.270,1.0,0.0754,0.008760,0.578000,0.0698,0.486,108.900,audio_features,spotify:track:0wJoRiX5K5BxlqZTolB2LD,https://api.spotify.com/v1/tracks/0wJoRiX5K5Bx...,https://api.spotify.com/v1/audio-analysis/0wJo...,170813.0,4.0,Hiphop,Purple Haze,NaN,NaN
58218,Jimi Hendrix,2aoo2jlRnM3A0NyLQqMN2f,0.438,0.805,8.0,-6.237,1.0,0.0624,0.002550,0.000057,0.0842,0.564,113.253,audio_features,spotify:track:2aoo2jlRnM3A0NyLQqMN2f,https://api.spotify.com/v1/tracks/2aoo2jlRnM3A...,https://api.spotify.com/v1/audio-analysis/2aoo...,240800.0,4.0,Underground Rap,All Along the Watchtower,NaN,NaN
62397,Al Green,63xdwScd1Ai1GigAwQxE8y,0.756,0.397,7.0,-10.701,0.0,0.0522,0.570000,0.011500,0.0542,0.507,203.970,audio_features,spotify:track:63xdwScd1Ai1GigAwQxE8y,https://api.spotify.com/v1/tracks/63xdwScd1Ai1...,https://api.spotify.com/v1/audio-analysis/63xd...,199396.0,4.0,Hiphop,Let's Stay Together,NaN,NaN
63422,Aerosmith,5MxNLUsfh7uzROypsoO5qe,0.307,0.433,1.0,-10.057,1.0,0.0290,0.388000,0.000091,0.3320,0.224,160.900,audio_features,spotify:track:5MxNLUsfh7uzROypsoO5qe,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,https://api.spotify.com/v1/audio-analysis/5MxN...,266960.0,4.0,Emo,Dream On,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670122,JoJo,3hc7PLcmuZbJGPjhIore1i,0.394,0.821,6.0,-4.304,0.0,0.0404,0.003020,0.019000,0.1860,0.509,165.528,audio_features,spotify:track:3hc7PLcmuZbJGPjhIore1i,https://api.spotify.com/v1/tracks/3hc7PLcmuZbJ...,https://api.spotify.com/v1/audio-analysis/3hc7...,223533.0,4.0,RnB,Too Little Too Late - 2018,NaN,NaN
670146,Simple Plan,1B8MHa0NveCF1ViBEOZWO5,0.522,0.751,1.0,-5.460,1.0,0.0435,0.000491,0.000002,0.1580,0.605,139.510,audio_features,spotify:track:1B8MHa0NveCF1ViBEOZWO5,https://api.spotify.com/v1/tracks/1B8MHa0NveCF...,https://api.spotify.com/v1/audio-analysis/1B8M...,232067.0,3.0,Emo,Addicted,NaN,NaN
670339,Ariana Grande,2vdBo4ALPYbHRUPKgtE5iC,0.287,0.364,9.0,-8.295,1.0,0.0306,0.418000,0.000018,0.1850,0.110,207.554,audio_features,spotify:track:2vdBo4ALPYbHRUPKgtE5iC,https://api.spotify.com/v1/tracks/2vdBo4ALPYbH...,https://api.spotify.com/v1/audio-analysis/2vdB...,271467.0,4.0,RnB,ghostin,NaN,NaN
670561,Puddle Of Mudd,7Ml1vXT6LaH34jVBGYygWY,0.406,0.932,6.0,-3.386,0.0,0.0936,0.000061,0.000000,0.0530,0.290,130.084,audio_features,spotify:track:7Ml1vXT6LaH34jVBGYygWY,https://api.spotify.com/v1/tracks/7Ml1vXT6LaH3...,https://api.spotify.com/v1/audio-analysis/7Ml1...,196293.0,4.0,Emo,Famous,NaN,NaN


In [259]:
artists_genres_consolidated = artists_exploded_enriched_nonnull.groupby('id')['genre'].apply(list).reset_index()

In [260]:
artists_genres_consolidated.head()

,id,genre
0,003FTlCpBTM4eSqYSWPv4H,[Emo]
1,00FROhC5g4iJdax5US8jRr,[Hiphop]
2,00GOPLxW4PGQuUYdPJh8K1,[Hiphop]
3,00Mb3DuaIH1kjrwOku9CGU,[Emo]
4,00NAQYOP4AmWR549nnYJZu,[RnB]


In [261]:
tracks_df = tracks_df.merge(artists_genres_consolidated, on = 'id',how = 'left')

In [262]:
tracks_df
#=tracks_df.drop(['genre_y','genre_x'],axis=1)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_upd3,names_strs,artists_song,genre
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,[Uli],[],[Uli],['Uli'],Carve,['Uli']Carve,NaN
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,[Fernando Pessoa],[],[Fernando Pessoa],['Fernando Pessoa'],Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa']Capítulo 2.16 - Banquero An...,NaN
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],Vivo para Quererte - Remasterizado,['Ignacio Corsini']Vivo para Quererte - Remast...,NaN
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],El Prisionero - Remasterizado,['Ignacio Corsini']El Prisionero - Remasterizado,NaN
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,[Dick Haymes],[],[Dick Haymes],['Dick Haymes'],Lady of the Evening,['Dick Haymes']Lady of the Evening,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526590,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,[阿YueYue],[],[阿YueYue],['阿YueYue'],云与海,['阿YueYue']云与海,NaN
526591,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,[ROLE MODEL],[],[ROLE MODEL],['ROLE MODEL'],blind,['ROLE MODEL']blind,NaN
526592,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,[FINNEAS],[],[FINNEAS],['FINNEAS'],What They'll Say About Us,['FINNEAS']What They'll Say About Us,NaN
526593,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,"[Gentle Bones, Clara Benin]",[],"[Gentle Bones, Clara Benin]","['Gentle Bones', 'Clara Benin']",A Day At A Time,"['Gentle Bones', 'Clara Benin']A Day At A Time",NaN


In [263]:
tracks_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_upd3,names_strs,artists_song,genre
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,[Uli],[],[Uli],['Uli'],Carve,['Uli']Carve,NaN
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,[Fernando Pessoa],[],[Fernando Pessoa],['Fernando Pessoa'],Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa']Capítulo 2.16 - Banquero An...,NaN
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],Vivo para Quererte - Remasterizado,['Ignacio Corsini']Vivo para Quererte - Remast...,NaN
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],El Prisionero - Remasterizado,['Ignacio Corsini']El Prisionero - Remasterizado,NaN
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,[Dick Haymes],[],[Dick Haymes],['Dick Haymes'],Lady of the Evening,['Dick Haymes']Lady of the Evening,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526590,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,[阿YueYue],[],[阿YueYue],['阿YueYue'],云与海,['阿YueYue']云与海,NaN
526591,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,[ROLE MODEL],[],[ROLE MODEL],['ROLE MODEL'],blind,['ROLE MODEL']blind,NaN
526592,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,[FINNEAS],[],[FINNEAS],['FINNEAS'],What They'll Say About Us,['FINNEAS']What They'll Say About Us,NaN
526593,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,"[Gentle Bones, Clara Benin]",[],"[Gentle Bones, Clara Benin]","['Gentle Bones', 'Clara Benin']",A Day At A Time,"['Gentle Bones', 'Clara Benin']A Day At A Time",NaN


In [264]:
tracks_df['year'] = tracks_df['release_date'].apply(lambda x: x.split('-')[0])

In [265]:
tracks_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_upd3,names_strs,artists_song,genre,year
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,[Uli],[],[Uli],['Uli'],Carve,['Uli']Carve,NaN,1922
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,[Fernando Pessoa],[],[Fernando Pessoa],['Fernando Pessoa'],Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa']Capítulo 2.16 - Banquero An...,NaN,1922
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],Vivo para Quererte - Remasterizado,['Ignacio Corsini']Vivo para Quererte - Remast...,NaN,1922
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],El Prisionero - Remasterizado,['Ignacio Corsini']El Prisionero - Remasterizado,NaN,1922
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,[Dick Haymes],[],[Dick Haymes],['Dick Haymes'],Lady of the Evening,['Dick Haymes']Lady of the Evening,NaN,1922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526590,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,[阿YueYue],[],[阿YueYue],['阿YueYue'],云与海,['阿YueYue']云与海,NaN,2020
526591,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,[ROLE MODEL],[],[ROLE MODEL],['ROLE MODEL'],blind,['ROLE MODEL']blind,NaN,2020
526592,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,[FINNEAS],[],[FINNEAS],['FINNEAS'],What They'll Say About Us,['FINNEAS']What They'll Say About Us,NaN,2020
526593,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,"[Gentle Bones, Clara Benin]",[],"[Gentle Bones, Clara Benin]","['Gentle Bones', 'Clara Benin']",A Day At A Time,"['Gentle Bones', 'Clara Benin']A Day At A Time",NaN,2021


In [266]:
float_cols = tracks_df.dtypes[tracks_df.dtypes == 'float64'].index.values

In [267]:
ohe_cols = 'popularity'

In [268]:
# rank with 5 buckets to normalize data
tracks_df['popularity_red'] = tracks_df['popularity'].apply(lambda x: int(x/5))


In [269]:
tracks_df['genre'] = tracks_df['genre'].apply(lambda d: d if isinstance(d, list) else [])

In [270]:
tracks_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_upd3,names_strs,artists_song,genre,year,popularity_red
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,[Uli],[],[Uli],['Uli'],Carve,['Uli']Carve,[],1922,1
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1,[Fernando Pessoa],[],[Fernando Pessoa],['Fernando Pessoa'],Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa']Capítulo 2.16 - Banquero An...,[],1922,0
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],Vivo para Quererte - Remasterizado,['Ignacio Corsini']Vivo para Quererte - Remast...,[],1922,0
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,[Ignacio Corsini],[],[Ignacio Corsini],['Ignacio Corsini'],El Prisionero - Remasterizado,['Ignacio Corsini']El Prisionero - Remasterizado,[],1922,0
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,[Dick Haymes],[],[Dick Haymes],['Dick Haymes'],Lady of the Evening,['Dick Haymes']Lady of the Evening,[],1922,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526590,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4,[阿YueYue],[],[阿YueYue],['阿YueYue'],云与海,['阿YueYue']云与海,[],2020,10
526591,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,[ROLE MODEL],[],[ROLE MODEL],['ROLE MODEL'],blind,['ROLE MODEL']blind,[],2020,14
526592,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,[FINNEAS],[],[FINNEAS],['FINNEAS'],What They'll Say About Us,['FINNEAS']What They'll Say About Us,[],2020,14
526593,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4,"[Gentle Bones, Clara Benin]",[],"[Gentle Bones, Clara Benin]","['Gentle Bones', 'Clara Benin']",A Day At A Time,"['Gentle Bones', 'Clara Benin']A Day At A Time",[],2021,11


In [271]:

def ohe_prep(df, column, new_name): 

    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [275]:
#function to build entire feature set
def create_feature_set(df, float_cols):
   
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genre'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = ohe_prep(df, 'year','year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)
     
    #add song id
    final['id']=df['id'].values
    
    return final

In [276]:
complete_feature_set = create_feature_set(tracks_df, float_cols=float_cols)#.mean(axis = 0)

In [277]:
complete_feature_set.head()

,genre|dark,genre|dnb,genre|emo,genre|hardstyle,genre|hiphop,genre|metal,genre|pop,genre|psytrance,genre|rap,genre|rnb,genre|techhouse,genre|techno,genre|trance,genre|trap,genre|underground,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,pop|0,pop|1,pop|2,pop|3,pop|4,pop|5,pop|6,pop|7,pop|8,pop|9,pop|10,pop|11,pop|12,pop|13,pop|14,pop|15,pop|16,pop|17,pop|18,pop|19,pop|20,year|1900,year|1922,year|1923,year|1924,year|1925,year|1926,year|1927,year|1928,year|1929,year|1930,year|1931,year|1932,year|1933,year|1934,year|1935,year|1936,year|1937,year|1938,year|1939,year|1940,year|1941,year|1942,year|1943,year|1944,year|1945,year|1946,year|1947,year|1948,year|1949,year|1950,year|1951,year|1952,year|1953,year|1954,year|1955,year|1956,year|1957,year|1958,year|1959,year|1960,year|1961,year|1962,year|1963,year|1964,year|1965,year|1966,year|1967,year|1968,year|1969,year|1970,year|1971,year|1972,year|1973,year|1974,year|1975,year|1976,year|1977,year|1978,year|1979,year|1980,year|1981,year|1982,year|1983,year|1984,year|1985,year|1986,year|1987,year|1988,year|1989,year|1990,year|1991,year|1992,year|1993,year|1994,year|1995,year|1996,year|1997,year|1998,year|1999,year|2000,year|2001,year|2002,year|2003,year|2004,year|2005,year|2006,year|2007,year|2008,year|2009,year|2010,year|2011,year|2012,year|2013,year|2014,year|2015,year|2016,year|2017,year|2018,year|2019,year|2020,year|2021,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.130172,0.08900,0.142750,0.092894,0.135341,0.14880,0.0302,0.0254,0.085113,0.00,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35iwgR4jXetI318WEWsa1Q
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.140262,0.05260,0.115835,0.197116,0.160040,0.00000,0.0296,0.1310,0.082806,0.15,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,021ht4sdgPcrDgSk7JTbKY
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087588,0.03540,0.118759,0.010546,0.199598,0.00436,0.0424,0.0914,0.105867,0.15,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,07A5yehtSnoedViJAZkNnc
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.064783,0.01892,0.098015,0.010381,0.199799,0.18360,0.0208,0.0794,0.137981,0.15,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

### Connection to Spotify API

In [281]:
#client id and secret for my application
client_id = id
client_secret= id

In [282]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [283]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [284]:
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://localhost:8834/')


In [285]:
sp = spotipy.Spotify(auth=token)

In [286]:
#gather playlist names and images- probably won't use images

id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [287]:
id_name

{'stop being mentally ill 😃': '1Ax0ss6gBDTy0EEX3XwuHz',
 'white woman victim mentalility': '0CqBqQDeemcY1uPno55gw3',
 'testing 123': '1kPx9eumti8hVG0SRfI68r',
 'it’s giving pregame ': '06uH5504qeZ9S1t8Yi98Re',
 "god's favorites": '34JjmLf1swnMZQdbGi1YqY',
 'r u from new york even ': '3NHuIVmGx1rprbzvAcxFZf',
 'tummy ache ': '5PTiaF2o9zptowGbU6SqhV',
 'march-ish': '7cEfzMNky6frWSZYPlMjII',
 'yer ': '7v9rp09s1WOdni2lWetiRv',
 'sleep': '1WED6iXU4UHvKKm2l2CnRI',
 'die for my bitch!!!!!!!!': '5S9fYTKASPBddrjIBi7XS2',
 'chilxz': '3vQlFH3Tw5RPsxvIVjNoR2',
 'coding music ': '6irm6oF0AMHMNUXDnoZZCL',
 'spring ': '2Q5l8i8D6qoT2ID2L1uNIV',
 'study new ': '7rkwo8iWk631naUIVe94fy',
 'jeannie <3': '04KEaRrMoDMi6iNDt6HN5b',
 'hmwrk': '5DXemOwm6LmS6xFWCVOA3R',
 'different state of mind ': '4K46i5JvI82UFLSaAs3gRH',
 'real?': '0OY59cJNNCCFaLVobRaCas',
 '888': '2PTUB2vwKNf4X6MfoVZ6oq',
 'too good to gatekeep ': '5VlWJBuXkhkI3TddxpDFqa',
 'viv ❤️\u200d🔥': '6Bu6IMLJZmQtEoTeI7QN01',
 'fug': '4DZjwi83plE5iMq

### Recommendation Set Up (vectorization, etc.)

In [288]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 

    params:
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
    
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [289]:
playlist_sleep = create_necessary_outputs('sleep', id_name,tracks_df)
playlist_playa = create_necessary_outputs('playa',id_name, tracks_df)
playlist_hw = create_necessary_outputs('hmwrk',id_name, tracks_df)

In [290]:
playlist_playa

,artist,name,id,url,date_added
95,Wallows,Remember When,5naar7XewEOAjOywIp6Jjq,https://i.scdn.co/image/ab67616d00001e0284feca...,2021-06-11 21:22:28+00:00
94,Dayglow,Hot Rod,0zePbRMJ9sd7wsZRlbPQua,https://i.scdn.co/image/ab67616d00001e02ccd63a...,2021-06-11 21:22:23+00:00
93,Dominic Fike,Babydoll,7yNf9YjeO5JXUE3JEBgnYc,https://i.scdn.co/image/ab67616d00001e027b1b6f...,2021-06-11 21:22:21+00:00
89,Phony Ppl,Why iii Love The Moon.,2x5Frs6ts14U4MSLrQQlXg,https://i.scdn.co/image/ab67616d00001e02d9ac65...,2021-06-11 05:21:35+00:00
88,WILLOW,Wait a Minute!,0y60itmpH0aPKsFiGxmtnh,https://i.scdn.co/image/ab67616d00001e026ee651...,2021-06-11 05:21:07+00:00
87,Steve Lacy,Dark Red,37y7iDayfwm3WXn5BiAoRk,https://i.scdn.co/image/ab67616d00001e02263886...,2021-06-11 05:21:00+00:00
86,Clairo,Flaming Hot Cheetos,1F6IbA7di42uPc3cff8PXV,https://i.scdn.co/image/ab67616d00001e0290c00a...,2021-06-11 05:20:56+00:00
84,Childish Gambino,Sober,5NhlpQ6BOIz3S5welptk1W,https://i.scdn.co/image/ab67616d00001e02720659...,2021-06-11 05:20:32+00:00
83,Frank Ocean,Pink + White,3xKsf9qdS1CyvXSMEid6g8,https://i.scdn.co/image/ab67616d00001e02c5649a...,2021-06-11 05:20:29+00:00
82,Harry Styles,Adore You,3jjujdWJ72nww5eGnfs2E7,https://i.scdn.co/image/ab67616d00001e0277fdcf...,2021-06-11 05:20:23+00:00


In [291]:
playlist_sleep

,artist,name,id,url,date_added
23,Billie Eilish,listen before i go,0tMSssfxAL2oV8Vri0mFHE,https://i.scdn.co/image/ab67616d00001e0250a314...,2022-02-21 08:32:15+00:00
22,Billie Eilish,i love you,6CcJMwBtXByIz4zQLzFkKc,https://i.scdn.co/image/ab67616d00001e0250a314...,2022-02-21 08:31:54+00:00
21,Billie Eilish,when the party's over,43zdsphuZLzwA9k4DJhU0I,https://i.scdn.co/image/ab67616d00001e0250a314...,2022-02-21 08:31:49+00:00
20,Billie Eilish,idontwannabeyouanymore,41zXlQxzTi6cGAjpOXyLYH,https://i.scdn.co/image/ab67616d00001e02a9f6c0...,2022-02-21 08:31:34+00:00
19,Rex Orange County,Pluto Projector,4EWBhKf1fOFnyMtUzACXEc,https://i.scdn.co/image/ab67616d00001e02733e6d...,2022-02-21 08:28:35+00:00
5,The Neighbourhood,Baby Came Home 2 / Valentines,40gvh6jP5qxMJuMmF9lG9A,https://i.scdn.co/image/ab67616d00001e02306658...,2022-02-21 08:15:22+00:00
3,Taylor Swift,mirrorball,0ZNU020wNYvgW84iljPkPP,https://i.scdn.co/image/ab67616d00001e0295f754...,2022-02-21 08:15:14+00:00


In [213]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    params: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [293]:
complete_feature_set_playlist_vector_sleep, complete_feature_set_nonplaylist_sleep = generate_playlist_feature(complete_feature_set, playlist_sleep, 1.09)


In [294]:
complete_feature_set_playlist_vector_sleep.shape

(146,)

In [295]:
complete_feature_set_playlist_vector_playa, complete_feature_set_nonplaylist_playa = generate_playlist_feature(complete_feature_set, playlist_playa, 1.09)


In [296]:
complete_feature_set_playlist_vector_playa.shape

(146,)

## song recommendations 

In [297]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [219]:
sleep_top40 = generate_playlist_recos(tracks_df, complete_feature_set_playlist_vector_sleep, complete_feature_set_nonplaylist_sleep)





In [220]:
sleep_top40

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_upd3,names_strs,artists_song,genre_x,year,popularity_red,genre_y,sim,url
86651,7F1CiKqrY44kh5cDqwHOnx,Love Is Gone - Acoustic,84,176972,0,"['SLANDER', 'Dylan Matthew']","['20DZAfCuP1TKZl5KcY7z3Q', '6d0ZjIp5L7Ygy2l02H...",2019-11-13,0.541,0.153,6,-10.654,1,0.0303,0.882,0.000002,0.1600,0.359,139.644,4,"[SLANDER, Dylan Matthew]",[],"[SLANDER, Dylan Matthew]","['SLANDER', 'Dylan Matthew']",Love Is Gone - Acoustic,"['SLANDER', 'Dylan Matthew']Love Is Gone - Aco...",[],2019,16,NaN,0.869998,https://i.scdn.co/image/ab67616d00001e023892a2...
86764,4ja2gzrNh9VNigzoXfmbwD,Mariposa,80,210000,0,['Peach Tree Rascals'],['0imE3buPhAowREqCrr4CYe'],2019-08-28,0.676,0.525,9,-5.880,1,0.0280,0.617,0.000000,0.3560,0.421,112.011,4,[Peach Tree Rascals],[],[Peach Tree Rascals],['Peach Tree Rascals'],Mariposa,['Peach Tree Rascals']Mariposa,[],2019,16,NaN,0.865005,https://i.scdn.co/image/ab67616d00001e02180bec...
86751,0ecC8p17cDNlxHXkuYqeR6,Surrender,80,183103,0,['Natalie Taylor'],['2WxjxdeF7GGdcCK276qViY'],2019-10-29,0.390,0.448,5,-8.570,0,0.0276,0.800,0.182000,0.1070,0.077,144.890,5,[Natalie Taylor],[],[Natalie Taylor],['Natalie Taylor'],Surrender,['Natalie Taylor']Surrender,[],2019,16,NaN,0.864713,https://i.scdn.co/image/ab67616d00001e02688ac6...
86678,2TH65lNHgvLxCKXM3apjxI,Callaita,82,250534,1,"['Bad Bunny', 'Tainy']","['4q3ewBCX7sLwd24euuV69X', '0GM7qgcRCORpGnfcN2...",2019-05-31,0.610,0.624,2,-4.773,1,0.3090,0.600,0.000002,0.2430,0.244,176.169,4,"[Bad Bunny, Tainy]",[],"[Bad Bunny, Tainy]","['Bad Bunny', 'Tainy']",Callaita,"['Bad Bunny', 'Tainy']Callaita",[],2019,16,NaN,0.862979,https://i.scdn.co/image/ab67616d00001e024aef42...
86716,1TQXIltqoZ5XXyfCbAeSQQ,Dancing With Your Ghost,81,197732,0,['Sasha Sloan'],['4xnihxcoXWK3UqryOSnbw5'],2019-06-28,0.629,0.327,11,-8.285,1,0.0290,0.761,0.000030,0.1030,0.381,74.953,4,[Sasha Sloan],[],[Sasha Sloan],['Sasha Sloan'],Dancing With Your Ghost,['Sasha Sloan']Dancing With Your Ghost,[],2019,16,NaN,0.862877,https://i.scdn.co/image/ab67616d00001e02141cf7...
86643,3H3r2nKWa3Yk5gt8xgmsEt,This City,84,194853,0,['Sam Fischer'],['6L1XC7NrmgWRlwAeLJvVtA'],2019-12-11,0.605,0.377,11,-6.213,1,0.0443,0.733,0.000000,0.0808,0.519,73.877,4,[Sam Fischer],[],[Sam Fischer],['Sam Fischer'],This City,['Sam Fischer']This City,[],2019,16,NaN,0.860396,https://i.scdn.co/image/ab67616d00001e0270e6c7...
86746,4SSnFejRGlZikf02HLewEF,bury a friend,80,193143,0,['Billie Eilish'],['6qqNVTkY8uBg9cP3Jd7DAH'],2019-03-29,0.905,0.389,8,-14.505,1,0.3320,0.740,0.162000,0.1060,0.196,120.046,4,[Billie Eilish],[],[Billie Eilish],['Billie Eilish'],bury a friend,['Billie Eilish']bury a friend,[],2019,16,NaN,0.855724,https://i.scdn.co/image/ab67616d00001e0250a314...
86707,0t3ZvGKlmYmVsDzBJAXK8C,Goodbyes (feat. Young Thug),81,174853,1,"['Post Malone', 'Young Thug']","['246dkjvS1zLTtiykXe5h60', '50co4Is1HCEo8bhOyU...",2019-09-06,0.548,0.653,5,-3.875,1,0.0810,0.456,0.000000,0.1030,0.163,150.068,4,"[Post Malone, Young Thug]",[],"[Post Malone, Young Thug]","['Post Malone', 'Young Thug']",Goodbyes (feat. Young Thug),"['Post Malone', 'Young Thug']Goodbyes (feat. Y...",[],2019,16,NaN,0.852133,https://i.scdn.co/image/ab67616d00001e029478c8...
86702,6Qs4SXO9dwPj5GKvVOv8Ki,Dancing With A Stranger (with Normani),81,171030,0,"['Sam Smith', 'Normani']","['2wY79sveU1sp5g7SokKOiI', '2cWZOOzeOm4WmBJRnD...",2019-01-11,0.741,0.520,8,-7.513,1,0.0656,0.450,0.000002,0.2220,0.347,102.998,4,"[Sam Smith, Normani]",[],"[Sam Smith, Normani]","['Sam Smith', 'Normani']",Dancing With A Stranger (with Normani),"['Sam Smith', 'Normani']Dancing With A Strange...",[],2019,16,NaN,0.851991,https://i.scdn.co/image/ab67616d00001e023b52ec...
86675,2HYvkdq6lpFXkVz9Tfqi8r,Big Gangsta,83,243463,1,['Kevin Gates'],['1gPhS1zisyXr5dHTYZyiMe'],2019-05-31,0.879,0.

In [298]:
playa_top40 = generate_playlist_recos(tracks_df, complete_feature_set_playlist_vector_playa, complete_feature_set_nonplaylist_playa)





In [299]:
playa_top40

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_upd3,names_strs,artists_song,genre,year,popularity_red,sim,url
85144,0Gx4VrHzS7pZOEAGrmXXBH,La Bicicleta,74,229510,0,"['Carlos Vives', 'Shakira']","['4vhNDa5ycK0ST968ek7kRr', '0EmeFodog0BfCgMzAI...",2017-11-10,0.745,0.944,0,-3.959,1,0.1340,0.2100,0.000001,0.3330,0.951,179.991,4,"[Carlos Vives, Shakira]",[],"[Carlos Vives, Shakira]","['Carlos Vives', 'Shakira']",La Bicicleta,"['Carlos Vives', 'Shakira']La Bicicleta",[],2017,14,0.750769,https://i.scdn.co/image/ab67616d00001e02575fb6...
85393,2Q85xwXagilpHKTfdrbC7M,La Rompe Corazones,72,204520,0,"['Daddy Yankee', 'Ozuna']","['4VMYDCV2IEDYJArk749S6m', '1i8SpTcr7yvPOmcqrb...",2017-01-13,0.685,0.859,5,-2.595,0,0.0553,0.1160,0.000000,0.3170,0.891,176.018,4,"[Daddy Yankee, Ozuna]",[],"[Daddy Yankee, Ozuna]","['Daddy Yankee', 'Ozuna']",La Rompe Corazones,"['Daddy Yankee', 'Ozuna']La Rompe Corazones",[],2017,14,0.739582,https://i.scdn.co/image/ab67616d00001e02396586...
85132,0pSBuHjILhNEo55xK1zrRt,Rolex,74,238587,0,['Ayo & Teo'],['27mFvqQj8KpjmdKIqcw1mG'],2017-03-15,0.804,0.886,1,-2.512,1,0.0400,0.0837,0.000000,0.2660,0.789,144.946,4,[Ayo & Teo],[],[Ayo & Teo],['Ayo & Teo'],Rolex,['Ayo & Teo']Rolex,[],2017,14,0.737295,https://i.scdn.co/image/ab67616d00001e0279dbd9...
164289,6DsPM0HMAXN5x4pzBR6bbo,Aftershock,65,230400,0,"['Macky Gee', 'Stuart Rowe']","['5EDztvTkyoKtK0ZdEdhOFx', '2qwGUda21fiqR1wm69...",2017-08-07,0.688,0.922,6,0.444,0,0.0362,0.2630,0.030200,0.1860,0.695,175.054,4,"[Macky Gee, Stuart Rowe]",[],"[Macky Gee, Stuart Rowe]","['Macky Gee', 'Stuart Rowe']",Aftershock,"['Macky Gee', 'Stuart Rowe']Aftershock",[],2017,13,0.734932,https://i.scdn.co/image/ab67616d00001e02d16080...
85155,1m69ELEgE6k5ZWsap40ozt,Se Preparó,74,188080,0,['Ozuna'],['1i8SpTcr7yvPOmcqrbnVXY'],2017-08-25,0.739,0.821,1,-4.233,1,0.0852,0.2220,0.000010,0.3110,0.693,171.077,4,[Ozuna],[],[Ozuna],['Ozuna'],Se Preparó,['Ozuna']Se Preparó,[],2017,14,0.732087,https://i.scdn.co/image/ab67616d00001e02b0a875...
85623,70lnL3QaSOIIyMa2X9aVRL,Perro Fiel (feat. Nicky Jam),70,195213,0,"['Shakira', 'Nicky Jam']","['0EmeFodog0BfCgMzAIvKQp', '1SupJlEpv7RS2tPNRa...",2017-05-26,0.750,0.760,1,-4.836,1,0.1980,0.1870,0.000000,0.0548,0.893,183.817,4,"[Shakira, Nicky Jam]",[],"[Shakira, Nicky Jam]","['Shakira', 'Nicky Jam']",Perro Fiel (feat. Nicky Jam),"['Shakira', 'Nicky Jam']Perro Fiel (feat. Nick...",[],2017,14,0.731973,https://i.scdn.co/image/ab67616d00001e02d05d3a...
146659,1WoDc1x28zHmxnPvvM5GLh,La Asesina,65,204733,0,['Zacarias Ferreira'],['1LKPL2O3vA3ozNsmshDg3o'],2017-09-27,0.810,0.803,3,-3.042,1,0.0349,0.3850,0.000000,0.1400,0.883,130.049,4,[Zacarias Ferreira],[],[Zacarias Ferreira],['Zacarias Ferreira'],La Asesina,['Zacarias Ferreira']La Asesina,[],2017,13,0.731828,https://i.scdn.co/image/ab67616d00001e024cdc89...
85539,4qknM1pQz53QOyfDVTjcM9,Me Enamoré,70,226893,0,['Shakira'],['0EmeFodog0BfCgMzAIvKQp'],2017-05-26,0.740,0.879,9,-2.251,0,0.1050,0.1930,0.000082,0.0928,0.948,103.710,4,[Shakira],[],[Shakira],['Shakira'],Me Enamoré,['Shakira']Me Enamoré,[],2017,14,0.730764,https://i.scdn.co/image/ab67616d00001e02d05d3a...
85134,6rPO02ozF3bM7NnOV4h6s2,Despacito - Remix,74,228827,0,"['Luis Fonsi', 'Daddy Yankee', 'Justin Bieber']","['4V8Sr092TqfHkfAA5fXXqG', '4VMYDCV2IEDYJArk74...",2017-04-17,0.653,0.816,2,-4.353,1,0.1670,0.2280,0.000000,0.0967,0.816,178.085,4,"[Luis Fonsi, Daddy Yankee, Justin Bieber]",[],"[Luis Fonsi, Daddy Yankee, Justin Bieber]","['Luis Fonsi', 'Daddy Yankee', 'Justin Bieber']",Despacito - Remix,"['Luis Fonsi', 'Daddy Yankee', 'Justin Bieber'...",[],2017,14,0.728892,https://i.scdn.co/image/ab67616d00001e02a6a335...
219166,6yZbbWkQ0cIjNXhOaBAfWP,Pesao,68,208027,1,"['Santa Estilo', 'Cartel De Santa', 'Millonario']","['59XuCTeNNS0fVgBrFoPNfG', '07PdYoE4jVRF6Ut40G...",2017-09-0